In [ ]:
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive/')


In [ ]:
print("Tensorflow version " + tf.__version__)
import os
import pathlib
import PIL
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers

In [ ]:
from zipfile import ZipFile

train_file = 'drive/MyDrive/Datasets/traindata.zip'
test_file = 'drive/MyDrive/Datasets/test_data.zip'

with ZipFile(train_file, 'r') as zip:
  zip.extractall("train_data")
  
train_dir = os.path.join(os.path.dirname("train_data"), 'train_data')
data_dir = pathlib.Path(train_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

free = list(data_dir.glob('free/*'))

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64 

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    brightness_range=(0.4,1.6),
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=(BATCH_SIZE),
    subset='training',
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=(BATCH_SIZE),
    subset='validation',
    class_mode='categorical'
) 

In [ ]:
image_batch, label_batch = next(val_generator)
image_batch.shape, label_batch.shape

class_names = sorted(train_generator.class_indices.keys())

plt.figure(figsize=(20, 20))
for images in image_batch:
  for i in range(20):
    ax = plt.subplot(4, 5, i + 1)
    plt.imshow(image_batch[i]/255)
    plt.title(class_names[label_batch[i][1].astype("uint8")])
    plt.axis("off")


In [ ]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('lot_labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
import tensorflow_hub as hub
IMAGE_SHAPE = (224,244)
def build_model(num_classes=2):
  inputs = layers.Input(shape=(224,224,3))
  model = tf.keras.applications.MobileNetV3Small(include_top=False,minimalistic=True, input_tensor=inputs, weights="imagenet") 
  #image input needs to be between [0-255] else [-1,1] if inlcude_preprocessing is False 
  
  model.trainable = False
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.2
  x = layers.Dropout(rate=top_dropout_rate)(x)
  outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)


  model = tf.keras.Model(inputs, outputs, name="MobileNetV3Large")
  # optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
  # optimizer = tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
  model.compile(
      optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
  )
  return model



In [ ]:
checkpoint_path = "weights.{epoch:02d}-{val_loss:.2f}.hdf5"                             
cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, verbose=1, save_weights_only=True,
        period=1)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4, restore_best_weights=True)

rlrop = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 1, factor= 0.4, min_lr= 1e-15, verbose=1)


In [ ]:
epochs = 5  
n_classes = 2

model = build_model(2)

model.summary()

In [ ]:
model.load_weights('./final_weights.hdf5')

In [ ]:


history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=[early_stop, rlrop,cp_callback]
    )

In [ ]:
def plot_hist(hist):
    plt.suptitle('Loss and Accuracy Plots', fontsize=18)

    plt.subplot(1,2,1)
    plt.plot(hist.history['loss'], label='Training Loss')
    plt.plot(hist.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.xlabel('Number of epochs', fontsize=15)
    plt.ylabel('Loss', fontsize=15)

    plt.subplot(1,2,2)
    plt.plot(hist.history['accuracy'], label='Train Accuracy')
    plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.xlabel('Number of epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.show()

plot_hist(history)

In [ ]:
model.save_weights('./mobilenetv3_base_train.hdf5')

# Eval on test set

In [ ]:
test_file = 'drive/MyDrive/Datasets/test_data.zip'

with ZipFile(train_file, 'r') as zip:
  zip.extractall("test_dir")
  
test_dir = os.path.join(os.path.dirname("train_data"), 'test_dir')
data_dir = pathlib.Path(test_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=(BATCH_SIZE),
    class_mode='categorical'
)

In [ ]:
model.evaluate(test_gen)

# Fine Tuning

In [ ]:
model.trainable = True
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
  layer.trainable =  False

for layer in model.layers:
  if isinstance(layer, tf.keras.layers.BatchNormalization):
      layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
history = model.fit(
        train_generator,
        epochs=5,
        validation_data=val_generator,
        callbacks=[early_stop, rlrop,cp_callback]
    )

In [ ]:
model.save_weights('./mobilenetv3_small.hdf5')

In [ ]:
## Retraining
model.trainable = True
fine_tune_at = 140

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
  layer.trainable = True

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-7),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

history = model.fit(
        train_generator,
        epochs=5,
        validation_data=val_generator,
        callbacks=[early_stop, rlrop,cp_callback]
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save
with open("mobilenetv3.tflite", "wb") as file:
  file.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpy9zao__t/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(train_dir + '/*/*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    # image = tf.cast(image / 255., tf.float32) #no for mobilenetv3
    # print(image.shape)
    image = tf.expand_dims(image, 0)
    # print(image.shape)
    yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

with open('mobilenetv3_small2.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
batch_images, batch_labels = next(val_generator)

logits = model(batch_images)
prediction = np.argmax(logits, axis=1)
truth = np.argmax(batch_labels, axis=1)

keras_accuracy = tf.keras.metrics.Accuracy()
keras_accuracy(prediction, truth)

print("Raw model accuracy: {:.3%}".format(keras_accuracy.result()))

In [ ]:
batch_images, batch_labels = next(val_generator)
def set_input_tensor(interpreter, input):
  input_details = interpreter.get_input_details()[0]
  tensor_index = input_details['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  # Inputs for the TFLite model must be uint8, so we quantize our input data.
  # NOTE: This step is necessary only because we're receiving input data from
  # ImageDataGenerator, which rescaled all image data to float [0,1]. When using
  # bitmap inputs, they're already uint8 [0,255] so this can be replaced with:
  input_tensor[:, :] = input
  # scale, zero_point = input_details['quantization']
  # input_tensor[:, :] = np.uint8(input / scale + zero_point)

def classify_image(interpreter, input,c):
  set_input_tensor(interpreter, input)
  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  output = interpreter.get_tensor(output_details['index'])
  # Outputs from the TFLite model are uint8, so we dequantize the results:
  scale, zero_point = output_details['quantization']
  output = scale * (output - zero_point)
  top_1 = np.argmax(output)
  return top_1

interpreter = tf.lite.Interpreter('mobilenetv3_small2.tflite')
interpreter.allocate_tensors()

# Collect all inference predictions in a list
batch_prediction = []
batch_truth = np.argmax(batch_labels, axis=1)

c=0

for i in range(len(batch_images)):
  prediction = classify_image(interpreter, batch_images[i],c)
  c=c+1
  batch_prediction.append(prediction)

# Compare all predictions to the ground truth
tflite_accuracy = tf.keras.metrics.Accuracy()
tflite_accuracy(batch_prediction, batch_truth)
print("Quant TF Lite accuracy: {:.3%}".format(tflite_accuracy.result()))


Quant TF Lite accuracy: 96.875%


In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
! edgetpu_compiler mobilenetv3_small.tflite
! edgetpu_compiler mobilenetv3_small2.tflite


In [ ]:
from google.colab import files

files.download('mobilenetv3_small.tflite')
files.download('mobilenetv3_small_edgetpu.tflite')
files.download('mobilenetv3_small2.tflite')
files.download('mobilenetv3_small2_edgetpu.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>